In [0]:
from datetime import datetime
start_date = datetime.now()
start_date_str = start_date.strftime('%Y-%m-%d %H:%M:%S')

In [0]:
%sql
-- Add new tickers to silver table
INSERT INTO yahoo_finance.silver.ticker_silver (ticker, inserted_datetime)
SELECT DISTINCT
  fb.ticker,
  current_timestamp()
FROM yahoo_finance.bronze.finance_bronze fb
LEFT OUTER JOIN 
(SELECT l.ticker, MAX(l.last_loaded_date) AS last_loaded_date
  FROM yahoo_finance.processrunlogs.processrunlog l 
  WHERE l.processname = 'process_silver_data' AND l.status = 'Completed' AND l.last_loaded_date IS NOT NULL
  GROUP BY l.ticker) l2
WHERE fb.ticker NOT IN (
  SELECT ticker FROM yahoo_finance.gold.ticker_gold
) AND fb.Datetime > nvl(l2.last_loaded_date, '2025-01-01');

In [0]:
%sql
-- Convert string values to proper data types and insert newly loaded data into silver financial table
INSERT INTO yahoo_finance.silver.finance_silver (ticker, date_id, datetime, close, high, low, open, volume, inserted_datetime)
SELECT DISTINCT
  fb.ticker, 
  date_format(nvl(try_cast(fb.datetime as date), '19000101'), 'yyyyMMdd') AS date_id,
  nvl(try_cast(fb.datetime as timestamp), '1900-01-01') as datetime,
  nvl(try_cast(fb.close as double), 0) as close,
  nvl(try_cast(fb.high as double), 0) as high,
  nvl(try_cast(fb.low as double), 0) as low,
  nvl(try_cast(fb.open as double), 0) as open,
  nvl(try_cast(fb.volume as bigint), 0) as `volume`,
  current_timestamp() as inserted_datetime
FROM yahoo_finance.bronze.finance_bronze fb
LEFT OUTER JOIN 
 (SELECT l.ticker, cast(MAX(l.last_loaded_date) AS STRING) AS last_loaded_date
  FROM yahoo_finance.processrunlogs.processrunlog l 
  WHERE l.processname = 'process_silver_data' AND l.status = 'Completed' AND l.last_loaded_date IS NOT NULL
  GROUP BY l.ticker) l2
  ON fb.ticker = l2.ticker
WHERE date_format(nvl(try_cast(fb.datetime as date), '1900-01-01'), 'yyyy-MM-dd') > nvl(l2.last_loaded_date, '2025-01-01');


In [0]:
# Audit the process
#  Get before_count - count of log rows before insertion of new rows
before_count = spark.sql("""
SELECT COUNT(*) as cnt
FROM yahoo_finance.processrunlogs.processrunlog
WHERE processname = 'process_silver_data' AND status = 'Completed'
""").collect()[0]['cnt']

# Try inserting new log rows
query_insert = f"""
INSERT INTO yahoo_finance.processrunlogs.processrunlog 
(ticker, processname, last_loaded_date, startdate, enddate, status)
SELECT f_s.ticker, 'process_silver_data' AS processname, max(f_s.datetime) AS last_loaded_date, '{start_date_str}', current_timestamp(), 'Completed'
FROM yahoo_finance.silver.finance_silver f_s
LEFT OUTER JOIN 
  (SELECT l.ticker, MAX(l.last_loaded_date) AS last_loaded_date
   FROM yahoo_finance.processrunlogs.processrunlog l 
   WHERE l.processname = 'process_silver_data' AND l.status = 'Completed'
   GROUP BY l.ticker) l2
  ON f_s.ticker = l2.ticker
WHERE f_s.Datetime > nvl(l2.last_loaded_date, '2025-01-01')
GROUP BY f_s.ticker
"""
spark.sql(query_insert)

# Get after_count - count of log rows after insertion of new rows
after_count = spark.sql("""
SELECT COUNT(*) as cnt
FROM yahoo_finance.processrunlogs.processrunlog
WHERE processname = 'process_silver_data' AND status = 'Completed'
""").collect()[0]['cnt']

# Insert NULL row if before_count == after_count to log that no new financials were loaded
if before_count == after_count:
    query_null = f"""
    INSERT INTO yahoo_finance.processrunlogs.processrunlog 
    (ticker, processname, last_loaded_date, startdate, enddate, status)
    VALUES (NULL, 'process_silver_data', NULL, '{start_date_str}', current_timestamp(), 'Completed')
    """
    spark.sql(query_null)